<a href="https://colab.research.google.com/github/bhagu/1000-Days-Of-Code/blob/main/Preparing_Data_for_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing Data For Machine Learning - Pluralsight Course

#Understanding the Need for Data Preparation

##Module Focus
Insufficient Data;
Excessive or overly complex data;
Non-representative data, missing data, outlier;
Oversampling and understanding;
Overfitting and underfitting models;

### Insufficient Data
Leads to Overfitting/ Underfitting

No great solution for insufficient data

Simply need to find more data sources

Dealing with Small Datasets

Model complexity - Simpler model with fewer model parameters, Less susceptible to overfitting, e.g. Naive Bayes classifier, logistic regression, Use ensemble techniques

Transfer learning - the practice of reusing a trained neural network that solves a problem similar to yours, usually leaving the network architeture unchanged and re-using some of all of the model weights

Data augmentation - Increase the number of training samples, Perturbed images are a form of data augmentation, Scaling, rotation, affine transforms, Makes CNN training more robust

Synthetic data - Artificially generate samples which mimic real world data


###Too Much Data
Data might be excessive in 2 ways

- Curse of dimensionality: Too many columns/features
- Outdated historical data: Too many rows

Concept Drift: The relationship between features (X-variables) and labels (Y-variables) changes over time; ML models fail to keep up, and consequently their performance suffers

Outdated historical data: If not eliminated, leads to concept drift;
This is a serious issue in specific applications
- Financial trading

Usually require human expert to judge which rows to leave out

Curse of dimensionality: 2 specific problems arise when too much data is available
- Deciding which data is actually relevant
- Aggregating very low-level data into useful features

Easier problem to solve:
- Feature selection: Deciding which data is actually relevant
- Feautre engineering: Aggregating very low-level data into useful features
- Dimensionality reduction: Reduce complexity without losing information

Concept Hierarchy: A mapping that combines very low-level features (e.g. latitudes and longitudes) into more general, usable features (e.g. zip codes)




###Non-Representative Data
Data is inaccurate, small errors have significant impact

Account for data cleaning and processing time

Data is not representative of the real world i.e. biased

Leads to biased models that perform poorly in practice

Mitigate using oversampling and undersampling

###Duplicate Data
If data can be flagged as duplicate, problem relatively easy to solve
- Simply de-duplicate

Can be hard to identify in some applications
- Real-time streaming

###Missing Data and Outliers
Data cleaning procedures can help significantly mitigate the effects of
- Missing data
- Outliers

Missing Data
- Deletion
- Imputation

Deletion / Listwise Deletion: Delete an entire record (row) if a single value (column) is missing. Simple, but can lead to bias

- Most common method in practices
- Can reduce sample size significanlty
- If values are not missing at random, can intriduce significant bias

Imputation: Fill in missing values, rather than deleting records with missing values. Missing values are inferred from known data.

- Methods range from very simple to very complex
- Simplest method - Use column average
- Can interpolate from nearby values 
- Can even build model to predict missing values

Univariate Imputation: rely only on known values in same feature

Multivariate imputation: Use all known data to infer missing value
- Construct regression models from other columns to predict this column
- Iteratively repeat for all columns

Hot-deck Imputation:
- Sort records based on any criteria
- for each missing value, use immediately prior available value
- "Last Observation Carried Forward"
- For time-series data, equivalent to assuming no change since last measurement

Mean Substitution:
- For each missing value, substitute mean of all available values
- Has effect of weakening correlations between columns
- Can be problematic when bivariate analysis is required

Regression:
- Fit model to predict missing column based on other column values
- Tends to strengthen correlations
- Regression and mean substitution have complementary strengths

Outliers
- Identifying Outliers
  - Distance from Mean
  - Distance from fitted line
- Coping with Outliers
  - Drop
  - Cap/Floor
  - Set to mean

Identifying Outliers

More than 3 Standard deviation - Distance from mean

Might also be data points that do not fit into the same relationship as the rest of the data


Coping with Outliers
- Always start by scrutinizing outliers
- If erroneous observation
  - Drop if all attributes of that point are erroneous
  - Set to mean if only one attribute is erroneous
- If genuine, legitimate outlier
  - Leave as-is if model is not distorted
  - Cap/Floor if model is distorted
    - Need to first standardize data
    - Cap positive outliers to +3
    - Floor negative outliers to -3

###Oversampling and Undersampling
Techniques that intentionally add bias to the data in order to make it balanced

Balacing Datasets
- Oversampling uncommon x or y values
- Undersampling of common x or y values

Forcibly Balanced Datasets

Oversampling and undersampling tend to
- Reduce accuracy
- Increase precision and recall

Related techniques include
- Case studies
- Stratified sampling

###Overfitting and Underfitting
Overfitting
- Model has memorized the training data
- Low training error
- Doesn't work well in the real world
- High test error

Underfitting
- Model unable to capture relationships in data
- Performs poorly on the training data
- Model too 'simple' to be useful

#Implementing Data Cleaning and Transformation

##Handling Missing Values

In [30]:
import sklearn

import pandas as pd
import numpy as np

import datetime

In [31]:
print(sklearn.__version__)

0.22.2.post1


In [32]:
print(np.__version__)

1.19.5


In [33]:
print(pd.__version__)

1.1.5


In [34]:
automobile_df = pd.read_csv("cars.csv")

automobile_df.head()

,Model,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin,bore,stroke,compression-ratio
0,chevrolet chevelle malibu,18.0,8,307,130,3504,12,1970[1975],"US; Detroit, Michigan",3.47,2.68,9
1,buick skylark 320,15.0,8,350,165,3693,11.5,1970,US],3.47,2.68,?
2,plymouth satellite,18.0,?,318,150,3436,11,"1970, 1976",US,2.68,3.47,9
3,amc rebel sst,16.0,8,304,150,3433,12,1970,US,3.19,3.40,10
4,ford torino,17.0,8,302,140,3449,10.5,1970-1972,US],3.19,3.40,8


In [35]:
automobile_df.columns

Index(['Model', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
       'Acceleration', 'Year', 'Origin', 'bore', 'stroke',
       'compression-ratio'],
      dtype='object')

In [36]:
automobile_df.describe()

,MPG,Horsepower,bore,stroke
count,385.000000,394.000000,394.000000,394.000000
mean,23.551429,104.337563,3.305025,3.289289
std,7.821598,38.449600,0.270336,0.271165
min,9.000000,46.000000,2.540000,2.070000
25%,17.500000,75.000000,3.080000,3.150000
50%,23.000000,92.500000,3.270000,3.350000
75%,29.000000,125.000000,3.580000,3.400000
max,46.600000,230.000000,3.940000,4.170000


In [37]:
  automobile_df.shape

(394, 12)

In [38]:
automobile_df = automobile_df.replace('?', np.nan)

automobile_df.head()

,Model,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin,bore,stroke,compression-ratio
0,chevrolet chevelle malibu,18.0,8,307,130,3504,12,1970[1975],"US; Detroit, Michigan",3.47,2.68,9
1,buick skylark 320,15.0,8,350,165,3693,11.5,1970,US],3.47,2.68,NaN
2,plymouth satellite,18.0,NaN,318,150,3436,11,"1970, 1976",US,2.68,3.47,9
3,amc rebel sst,16.0,8,304,150,3433,12,1970,US,3.19,3.40,10
4,ford torino,17.0,8,302,140,3449,10.5,1970-1972,US],3.19,3.40,8


In [39]:
automobile_df.isna().sum()

Model                0
MPG                  9
Cylinders            2
Displacement         1
Horsepower           0
Weight               1
Acceleration         1
Year                 0
Origin               0
bore                 0
stroke               0
compression-ratio    2
dtype: int64

In [40]:
automobile_df['MPG'] = automobile_df['MPG'].fillna(automobile_df['MPG'].mean())

automobile_df.isna().sum()

Model                0
MPG                  0
Cylinders            2
Displacement         1
Horsepower           0
Weight               1
Acceleration         1
Year                 0
Origin               0
bore                 0
stroke               0
compression-ratio    2
dtype: int64

In [41]:
automobile_df = automobile_df.dropna()

In [42]:
 automobile_df.shape

(387, 12)

In [43]:
automobile_df.isna().sum()

Model                0
MPG                  0
Cylinders            0
Displacement         0
Horsepower           0
Weight               0
Acceleration         0
Year                 0
Origin               0
bore                 0
stroke               0
compression-ratio    0
dtype: int64

In [44]:
automobile_df.isnull().sum()

Model                0
MPG                  0
Cylinders            0
Displacement         0
Horsepower           0
Weight               0
Acceleration         0
Year                 0
Origin               0
bore                 0
stroke               0
compression-ratio    0
dtype: int64

In [45]:
automobile_df.sample(5)

,Model,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin,bore,stroke,compression-ratio
162,buick skyhawk,21.0,6,231,110,3039,15,1975,US,3.19,3.03,9
190,chevrolet nova,22.0,6,250,105,3353,14.5,1976,US,3.19,3.40,8.5
185,chevrolet chevelle malibu classic,17.5,8,305,140,4215,13,1976,US,3.19,3.40,9
386,toyota celica gt,32.0,4,144,96,2665,13.9,1982,Japan,3.15,3.29,9.4
365,chevrolet cavalier 2-door,34.0,4,112,88,2395,18,1982-1985,US,3.58,3.64,21.5


In [46]:
automobile_df.drop(['Model'], axis=1, inplace=True)

In [47]:
automobile_df.sample(5)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin,bore,stroke,compression-ratio
344,37.7,4,89,62,2050,17.3,1981,Japan,3.54,2.76,11.5
166,23.0,4,140,83,2639,17,1975,US,3.24,3.08,9.4
290,18.5,8,360,150,3940,13,1979,US,3.27,3.35,9.2
269,23.2,4,156,105,2745,16.7,1978,US,3.01,3.40,23
119,19.0,4,121,112,2868,15.5,1973,Europe,3.03,3.39,7.6


In [48]:
automobile_df.drop(['bore', 'stroke', 'compression-ratio'], axis=1, inplace=True)

automobile_df.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin
0,18.000000,8,307,130,3504,12,1970[1975],"US; Detroit, Michigan"
3,16.000000,8,304,150,3433,12,1970,US
4,17.000000,8,302,140,3449,10.5,1970-1972,US]
6,14.000000,8,454,220,4354,9,1970[1975],"US; Detroit, Michigan"
7,23.551429,8,440,215,4312,8.5,1970,US


##Cleaning Data

In [49]:
automobile_df['Year'].str.isnumeric().value_counts()

True     351
False     36
Name: Year, dtype: int64

In [50]:
automobile_df['Year'].loc[automobile_df['Year'].str.isnumeric() == False]

0          1970[1975]
4           1970-1972
6          1970[1975]
30      1971[1973]971
35         1971[1973]
40         1971[1973]
44         1971[1973]
49         1971[1973]
56         1972[1973]
61         1972[1973]
65         1972[1973]
68         1972[1973]
73         1972[1973]
75            1972-73
79            1972-73
82            1972-73
96          1973-1974
97         1973, 1974
102    1973, 19741973
105        1973, 1974
108        1973, 1974
112        1973, 1974
115        1973, 1974
126        1974, 1975
131        1974, 1975
134        1974, 1975
137        1974, 1975
139    1974, 19751974
142        1974, 1975
365         1982-1985
374         1982-1985
380         1982-1985
383         1982-1985
387         1982-1985
389         1982-1985
391         1982-1985
Name: Year, dtype: object

In [51]:
extr = automobile_df['Year'].str.extract(r'^(\d{4})', expand=False)

extr.head()

0    1970
3    1970
4    1970
6    1970
7    1970
Name: Year, dtype: object

In [52]:
automobile_df['Year'].isnull().values.any()

False

In [53]:
automobile_df['Year'] = pd.to_numeric(extr)
automobile_df['Year'].dtype

dtype('int64')

In [54]:
automobile_df.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin
0,18.000000,8,307,130,3504,12,1970,"US; Detroit, Michigan"
3,16.000000,8,304,150,3433,12,1970,US
4,17.000000,8,302,140,3449,10.5,1970,US]
6,14.000000,8,454,220,4354,9,1970,"US; Detroit, Michigan"
7,23.551429,8,440,215,4312,8.5,1970,US


In [55]:
automobile_df['Age'] = datetime.datetime.now().year - automobile_df['Year']

automobile_df.drop(['Year'], axis=1, inplace=True)

automobile_df.sample(5)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Origin,Age
259,18.1,6,258,120,3410,15.1,US,43
325,36.4,5,121,67,2950,19.9,Europe-Germany,41
184,27.0,4,101,83,2202,15.3,Europe,45
366,31.0,4,112,85,2575,16.2,US,39
337,23.5,6,173,110,2725,12.6,US,40


In [58]:
automobile_df.dtypes

MPG             float64
Cylinders        object
Displacement     object
Horsepower        int64
Weight           object
Acceleration     object
Origin           object
Age               int64
dtype: object

In [59]:
automobile_df['Cylinders'].isnull().values.any()

False

In [60]:
automobile_df['Cylinders'].str.isnumeric().value_counts()

True     378
False      9
Name: Cylinders, dtype: int64

In [61]:
automobile_df['Cylinders'].loc[automobile_df['Cylinders'].str.isnumeric() == False]

11    -
22    -
32    -
42    -
51    -
58    -
69    -
82    -
89    -
Name: Cylinders, dtype: object

In [63]:
cylinders = automobile_df['Cylinders'].loc[automobile_df['Cylinders'] != '-']

cmean = cylinders.astype(int).mean()

In [64]:
cmean

5.420634920634921

In [65]:
automobile_df['Cylinders'] = automobile_df['Cylinders'].replace('-', cmean).astype(int)

In [66]:
automobile_df.dtypes

MPG             float64
Cylinders         int64
Displacement     object
Horsepower        int64
Weight           object
Acceleration     object
Origin           object
Age               int64
dtype: object

In [67]:
automobile_df['Displacement'] = pd.to_numeric(automobile_df['Displacement'], errors='coerce')
automobile_df.dtypes

MPG             float64
Cylinders         int64
Displacement    float64
Horsepower        int64
Weight           object
Acceleration     object
Origin           object
Age               int64
dtype: object

In [68]:
automobile_df['Weight'] = pd.to_numeric(automobile_df['Weight'], errors='coerce')
automobile_df.dtypes

MPG             float64
Cylinders         int64
Displacement    float64
Horsepower        int64
Weight            int64
Acceleration     object
Origin           object
Age               int64
dtype: object

In [70]:
automobile_df['Acceleration'] = pd.to_numeric(automobile_df['Acceleration'], errors='coerce')
automobile_df.dtypes

MPG             float64
Cylinders         int64
Displacement    float64
Horsepower        int64
Weight            int64
Acceleration    float64
Origin           object
Age               int64
dtype: object

In [72]:
automobile_df['Origin'].head(15)

0     US; Detroit, Michigan
3                        US
4                       US]
6     US; Detroit, Michigan
7                        US
8                        US
9                       US]
10                       US
11                       US
12    US; Detroit, Michigan
13                       US
14             Japan; Aichi
15                       US
16                      US]
17                       US
Name: Origin, dtype: object

In [73]:
automobile_df['Origin'].unique()

array(['US; Detroit, Michigan', 'US', 'US]', 'Japan; Aichi',
       'Europe-Germany', 'Europe', 'Japan; Tokyo', 'Japan; Aichi]',
       'US; Detroit, Michigan]', 'Japan; Tokyo]', 'Japan',
       'Europe Germany'], dtype=object)

In [74]:
automobile_df['Origin'] = np.where(automobile_df['Origin'].str.contains('US'),
                                   'US',
                                   automobile_df['Origin'])

automobile_df['Origin'].unique()

array(['US', 'Japan; Aichi', 'Europe-Germany', 'Europe', 'Japan; Tokyo',
       'Japan; Aichi]', 'Japan; Tokyo]', 'Japan', 'Europe Germany'],
      dtype=object)

In [75]:
automobile_df['Origin'] = np.where(automobile_df['Origin'].str.contains('Japan'),
                                   'Japan',
                                   automobile_df['Origin'])

automobile_df['Origin'].unique()

array(['US', 'Japan', 'Europe-Germany', 'Europe', 'Europe Germany'],
      dtype=object)

In [76]:
automobile_df['Origin'] = np.where(automobile_df['Origin'].str.contains('Europe'),
                                   'Europe',
                                   automobile_df['Origin'])

automobile_df['Origin'].unique()

array(['US', 'Japan', 'Europe'], dtype=object)

In [77]:
automobile_df.describe()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
count,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000
mean,23.672514,5.410853,192.184755,103.645995,2965.387597,15.573643,44.917313
std,7.736579,1.667795,103.703706,38.128651,846.332848,2.746260,3.668715
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,39.000000
25%,17.600000,4.000000,102.500000,75.000000,2221.500000,13.900000,42.000000
50%,23.200000,4.000000,146.000000,92.000000,2790.000000,15.500000,45.000000
75%,29.000000,6.000000,260.000000,121.000000,3589.500000,17.050000,48.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,51.000000


In [78]:
automobile_df.to_csv('cars_processed.csv', index=False)